<a href="https://colab.research.google.com/github/z-arabi/pytorchTutorial/blob/master/09_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [35]:
!git clone https://github.com/z-arabi/pytorchTutorial.git
% cd pytorchTutorial
% pwd

Cloning into 'pytorchTutorial'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 112 (delta 45), reused 66 (delta 16), pack-reused 0
Receiving objects: 100% (112/112), 7.61 MiB | 29.73 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/pytorchTutorial/pytorchTutorial/pytorchTutorial/pytorchTutorial


'/content/pytorchTutorial/pytorchTutorial/pytorchTutorial/pytorchTutorial'

In [36]:
# we have two approaches to load data
# 1. load all of the data
# 2. load the size of batch in each time

# gradient computation etc. not efficient for whole data set
# -> divide dataset into small batches

'''
#training loop
in one epoch > we get the batch > train > the iteration finish > get another batch and train > epoch finish
for epoch in range(num_epochs):
    # loop over all batches
    for i in range(total_batches):
        batch_x, batch_y = ...
'''

# epoch = one forward and backward pass of ALL training samples
# batch_size = number of training samples used in one forward/backward pass
# number of iterations = number of passes, each pass (forward+backward) using [batch_size] number of sampes
# e.g : 100 samples, batch_size=20 -> 100/20=5 iterations for 1 epoch

# --> DataLoader can do the batch computation for us

# Implement a custom Dataset:
# inherit Dataset
# implement __init__ , __getitem__ , and __len__

'\n#training loop\nin one epoch > we get the batch > train > the iteration finish > get another batch and train > epoch finish\nfor epoch in range(num_epochs):\n    # loop over all batches\n    for i in range(total_batches):\n        batch_x, batch_y = ...\n'

In [37]:
# first just dive into that
xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
print(xy[0])
print(xy.shape)
n_samples = xy.shape[0]

x_data = torch.from_numpy(xy[:, 1:]) # size [n_samples, n_features]
y_data = torch.from_numpy(xy[:, [0]]) # size [n_samples, 1]

print(y_data.shape)

[1.000e+00 1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00
 3.060e+00 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
(178, 14)
torch.Size([178, 1])


In [38]:
class WineDataset(Dataset):

    def __init__(self):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        
        self.n_samples = xy.shape[0]

        # here the first column is the class label, the rest are the features
        self.x_data = torch.from_numpy(xy[:, 1:]) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(xy[:, [0]]) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



In [39]:
# create dataset object which has get and len items
dataset = WineDataset()

# get first sample and unpack
first_data = dataset[0]
features, labels = first_data
print(features, labels)
print(len(dataset))

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])
178


In [41]:
# Load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
# !!! IF YOU GET AN ERROR DURING LOADING, SET num_workers TO 0 !!!
train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=2)

# convert to an iterator and look at one random sample
# to have access to the datas > make iterable the data loader
# the output of iter is a list for data + a list for output == > like what we define in dataset
dataiter = iter(train_loader)
data = dataiter.next()
features, labels = data
print(features, labels)

print(len(train_loader))

*_, last = dataiter
print(last)

tensor([[1.3560e+01, 1.7300e+00, 2.4600e+00, 2.0500e+01, 1.1600e+02, 2.9600e+00,
         2.7800e+00, 2.0000e-01, 2.4500e+00, 6.2500e+00, 9.8000e-01, 3.0300e+00,
         1.1200e+03],
        [1.1030e+01, 1.5100e+00, 2.2000e+00, 2.1500e+01, 8.5000e+01, 2.4600e+00,
         2.1700e+00, 5.2000e-01, 2.0100e+00, 1.9000e+00, 1.7100e+00, 2.8700e+00,
         4.0700e+02],
        [1.3170e+01, 2.5900e+00, 2.3700e+00, 2.0000e+01, 1.2000e+02, 1.6500e+00,
         6.8000e-01, 5.3000e-01, 1.4600e+00, 9.3000e+00, 6.0000e-01, 1.6200e+00,
         8.4000e+02],
        [1.2370e+01, 1.6300e+00, 2.3000e+00, 2.4500e+01, 8.8000e+01, 2.2200e+00,
         2.4500e+00, 4.0000e-01, 1.9000e+00, 2.1200e+00, 8.9000e-01, 2.7800e+00,
         3.4200e+02]]) tensor([[1.],
        [2.],
        [3.],
        [2.]])
45
[tensor([[1.2510e+01, 1.7300e+00, 1.9800e+00, 2.0500e+01, 8.5000e+01, 2.2000e+00,
         1.9200e+00, 3.2000e-01, 1.4800e+00, 2.9400e+00, 1.0400e+00, 3.5700e+00,
         6.7200e+02],
        [1.2840e+0

In [42]:
print(round(13/4))
print(math.ceil(13/4))

3
4


In [44]:
# Dummy Training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # feed forward, update
        # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
        # Run your training process
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')


178 45
Epoch: 1/2, Step 5/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 10/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 15/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 20/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 25/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 30/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 35/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 40/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 45/45| Inputs torch.Size([2, 13]) | Labels torch.Size([2, 1])
Epoch: 2/2, Step 5/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 2/2, Step 10/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 2/2, Step 15/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 2/2, Step 20/45| Inputs torch.Size([4, 1

In [47]:
# some famous datasets are available in torchvision.datasets
# e.g. MNIST, Fashion-MNIST, CIFAR10, COCO
# we can pass transform argument to the available dataset > ToTensor > convert images + np arr to tensors

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=torchvision.transforms.ToTensor(),  
                                           download=True)

train_loader = DataLoader(dataset=train_dataset, 
                                           batch_size=3, 
                                           shuffle=True)

# look at one random sample
dataiter = iter(train_loader)
data = dataiter.next()
inputs, targets = data
print(inputs.shape, targets.shape)


torch.Size([3, 1, 28, 28]) torch.Size([3])
